In [1]:
#init
import torch, numpy, sys
sys.path.append("..")
from chatbot_end2end.module.interact_session import InteractSession
from nlptools.utils import Config
import torch.nn as nn

cfg = Config("../config/elsa.yml")
session = InteractSession.build(cfg)

/usr/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [4]:
skill_name = "cornell"
skill = session.topic_manager.skills[skill_name]
state_dict = skill.model.decoder.state_dict()
dialog_status = session.new_dialog()
dialog_status.add_utterance("I would hardly say you look like trash")
data = dialog_status.status2data()
data.to("cuda:0")

encoder_out = skill.model.dialog_embedding(data["utterance"].data, data["utterance_mask"].data, data["sentiment"].data)
utterance_mask = data["utterance_mask"].data
print(encoder_out.shape)
print(utterance_mask.shape)

torch.Size([1, 20, 768])
torch.Size([1, 20])


In [61]:
import nlptools.zoo.encoders.transformer
import chatbot_end2end.model.generative_tracker
import importlib
importlib.reload(nlptools.zoo.encoders.transformer)
importlib.reload(chatbot_end2end.model.generative_tracker)

from nlptools.zoo.encoders.transformer import TransformerDecoder
from chatbot_end2end.model.generative_tracker import Generative_Tracker

embedding = skill.model.encoder.embedding

decoder_hidden_layers = skill.model.config["decoder_hidden_layers"]
decoder_attention_heads = skill.model.config["decoder_attention_heads"]
decoder_hidden_size = skill.model.config["decoder_hidden_size"]
bert_model_name = skill.model.config["bert_model_name"]
pad_id = skill.model.pad_id
bos_id = skill.model.bos_id
eos_id = skill.model.eos_id
unk_id = skill.model.unk_id
beam_size = 5
len_penalty = 1
unk_penalty = 1
dropout=0

shared_layers = {"encoder": skill.model.encoder}

tracker = Generative_Tracker(shared_layers=shared_layers, skill_name=skill_name, bert_model_name=bert_model_name, decoder_hidden_laers=decoder_hidden_layers,
                             decoder_attention_heads=decoder_attention_heads, decider_hidden_size=decoder_hidden_size, dropout=dropout, pad_id=pad_id, bos_id=bos_id,
                            eos_id=eos_id, unk_id=unk_id, beam_size=beam_size, len_penalty=len_penalty, unk_penalty=unk_penalty)
tracker = tracker.to("cuda:0")

decoder = TransformerDecoder(embedding, decoder_hidden_layers, decoder_attention_heads, decoder_hidden_size, dropout)
decoder = decoder.to("cuda:0")
decoder.load_state_dict(state_dict)

tracker.decoder = decoder

tracker.beam_search(encoder_out, utterance_mask)

output_buf torch.Size([5, 1])
output_probs1 tensor([[[-22.7928, -22.5807, -22.7010,  ..., -21.8003, -22.2742, -23.3795]],

        [[-22.7928, -22.5807, -22.7010,  ..., -21.8003, -22.2742, -23.3795]],

        [[-22.7928, -22.5807, -22.7010,  ..., -21.8003, -22.2742, -23.3795]],

        [[-22.7928, -22.5807, -22.7010,  ..., -21.8003, -22.2742, -23.3795]],

        [[-22.7928, -22.5807, -22.7010,  ..., -21.8003, -22.2742, -23.3795]]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward>) torch.Size([5, 1, 30522])
output_probs2 tensor([[-22.7928, -22.5807, -22.7010,  ..., -21.8003, -22.2742, -23.3795],
        [-22.7928, -22.5807, -22.7010,  ..., -21.8003, -22.2742, -23.3795],
        [-22.7928, -22.5807, -22.7010,  ..., -21.8003, -22.2742, -23.3795],
        [-22.7928, -22.5807, -22.7010,  ..., -21.8003, -22.2742, -23.3795],
        [-22.7928, -22.5807, -22.7010,  ..., -21.8003, -22.2742, -23.3795]],
       device='cuda:0', grad_fn=<ViewBackward>) torch.Size([5, 30522])
prev_scores ten

(tensor([[  101, 28650, 28650, 28650, 28650, 28650, 28650, 28650, 28650, 28650,
          28650, 28650, 28650, 28650, 28650, 28650, 28650, 28650, 28650, 28650]],
        device='cuda:0'), array([-0.02742364]))

In [ ]:
tracker